In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

real_images = load_images_from_folder('/content/drive/MyDrive/real_and_fake_face/training_real')
fake_images = load_images_from_folder('/content/drive/MyDrive/real_and_fake_face/training_fake')


def preprocess_images(images):
    processed_images = []
    for img in images:
        img = cv2.resize(img, (224, 224))  # Resize to a common size
        img = img / 255.0  # Normalize pixel values to [0, 1]
        processed_images.append(img)
    return np.array(processed_images)

real_images = preprocess_images(real_images)
fake_images = preprocess_images(fake_images)


labels = [0] * len(real_images) + [1] * len(fake_images)
images = np.concatenate((real_images, fake_images), axis=0)


X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

X_train = np.array(X_train)
y_train = np.array(y_train)

model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification (real or fake)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)




Epoch 1/10
41/41 [==============================] - 72s 2s/step - loss: 1.6559 - accuracy: 0.5475 - val_loss: 0.6922 - val_accuracy: 0.5382
Epoch 2/10
41/41 [==============================] - 71s 2s/step - loss: 0.6792 - accuracy: 0.6080 - val_loss: 0.6807 - val_accuracy: 0.5719
Epoch 3/10
41/41 [==============================] - 71s 2s/step - loss: 0.6214 - accuracy: 0.6868 - val_loss: 0.6686 - val_accuracy: 0.6147
Epoch 4/10
41/41 [==============================] - 70s 2s/step - loss: 0.5114 - accuracy: 0.7588 - val_loss: 0.7465 - val_accuracy: 0.5902
Epoch 5/10
41/41 [==============================] - 72s 2s/step - loss: 0.3266 - accuracy: 0.8729 - val_loss: 0.8766 - val_accuracy: 0.5719
Epoch 6/10
41/41 [==============================] - 72s 2s/step - loss: 0.1782 - accuracy: 0.9395 - val_loss: 1.3140 - val_accuracy: 0.5413
Epoch 7/10
41/41 [==============================] - 70s 2s/step - loss: 0.0865 - accuracy: 0.9816 - val_loss: 1.4388 - val_accuracy: 0.5382
Epoch 8/10
41/41 [==

In [2]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

def predict_image(image_path):

    img = cv2.imread(image_path)

    if img is None:
        return "Error: Unable to load the image."

    if img.shape[0] == 0 or img.shape[1] == 0:
        return "Error: Invalid image dimensions."

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))

    if img is None:
        return "Error: Unable to resize the image."

    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)

    if prediction[0] < 0.5:
        return "Real"
    else:
        return "Fake"


13/13 [==============================] - 5s 402ms/step - loss: 2.1290 - accuracy: 0.5892


In [ ]:
# from google.colab import files

# files.upload()

result = predict_image('/WhatsApp Image 2023-09-23 at 3.54.28 PM.jpeg')
print(f"The image is {result}")

1/1 [==============================] - 0s 47ms/step
The image is Fake
